In [1]:
#cnn : 1 卷积
# ABC 
# A: 激励函数+矩阵 乘法加法
# A CNN :  pool（激励函数+矩阵 卷积 加法）
# C：激励函数+矩阵 乘法加法（A-》B）
# C：激励函数+矩阵 乘法加法（A-》B） + softmax（矩阵 乘法加法）
# loss：tf.reduce_mean(tf.square(y-layer2))
# loss：code

# 1 import 
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# 2 load data
mnist = input_data.read_data_sets('MNIST_data',one_hot = True)

# 3 input
imageInput = tf.placeholder(tf.float32,[None,784]) # 28*28 
labeInput = tf.placeholder(tf.float32,[None,10]) # knn

# 4 data reshape 完成维度调整
# [None,784]->M*28*28*1  2D->4D  28*28 wh 1 channel 
imageInputReshape = tf.reshape(imageInput,[-1,28,28,1])

# 5 卷积 w0 : 卷积内核 5*5 out:32  in:1 
# 生成正态分布数据的维度，方差
w0 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev = 0.1))
b0 = tf.Variable(tf.constant(0.1,shape=[32]))

# 6 # layer1：激励函数+卷积运算
# imageInputReshape : M*28*28*1  w0:5,5,1, 32  
layer1 = tf.nn.relu(tf.nn.conv2d(imageInputReshape,w0,strides=[1,1,1,1],padding='SAME')+b0)
# 输出大小M*28*28*32
# pool层 下采样 数据量减少很多M*28*28*32  对应除ksize => 池化层输出M*7*7*32
layer1_pool = tf.nn.max_pool(layer1,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')
# [1 2 3 4]->[4] 4来自于这四个数中最大值

# 7 layer2 out : 激励函数+乘加运算； 卷积: softmax(回归计算，计算输出值)（激励函数 + 乘加运算）
# [7*7*32,1024]
w1 = tf.Variable(tf.truncated_normal([7*7*32,1024],stddev=0.1))
b1 = tf.Variable(tf.constant(0.1,shape=[1024]))
# 维度转换，四维数据转换为2维的
h_reshape = tf.reshape(layer1_pool,[-1,7*7*32])# M*7*7*32 -> N*N1


# [N*7*7*32]  [7*7*32,1024] = N*1024
h1 = tf.nn.relu(tf.matmul(h_reshape,w1)+b1)
# 7.1 softMax
w2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
b2 = tf.Variable(tf.constant(0.1,shape=[10]))
pred = tf.nn.softmax(tf.matmul(h1,w2)+b2)# N*1024  1024*10 = N*10
# N*10( 概率 )N1【0.1 0.2 0.4 0.1 0.2 。。。】
# label。        【0 0 0 0 1 0 0 0.。。】
loss0 = labeInput*tf.log(pred)
loss1 = 0
# 7.2 
for m in range(0,500):#  test 100
    for n in range(0,10):
        loss1 = loss1 - loss0[m,n]
loss = loss1/500

# 8 train
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
# 9 run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        images,labels = mnist.train.next_batch(500)
        sess.run(train,feed_dict={imageInput:images,labeInput:labels})
        
        pred_test = sess.run(pred,feed_dict={imageInput:mnist.test.images,labeInput:labels})
        acc = tf.equal(tf.arg_max(pred_test,1),tf.arg_max(mnist.test.labels,1))
        acc_float = tf.reduce_mean(tf.cast(acc,tf.float32))
        acc_result = sess.run(acc_float,feed_dict={imageInput:mnist.test.images,labeInput:mnist.test.labels})
        print(acc_result)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, Variable/read)]]
	 [[Node: Softmax/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_17_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D', defined at:
  File "D:\softEnvDown\Anaconda2\envs\py3\lib\runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\softEnvDown\Anaconda2\envs\py3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\softEnvDown\Anaconda2\envs\py3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\softEnvDown\Anaconda2\envs\py3\